<a target="_blank" href="https://colab.research.google.com/github/sonder-art/fdd_o23/blob/main/codigo/polars/04_ejercicio_polars.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
pip install polars

In [2]:
import polars as pl
import random
import string


# Generacion de Datos

In [4]:
# Funciones auxiliares para generar datos
def random_string(length=5):
    letters = string.ascii_lowercase
    return ''.join(random.choice(letters) for i in range(length))

def generate_data(n=1000):
    data = {
        "id_venta": list(range(1, n + 1)),
        "producto": [random_string() for _ in range(n)],
        "cantidad": [random.randint(1, 20) for _ in range(n)],
        "precio_unitario": [random.uniform(10, 100) for _ in range(n)],
        # Use the correct way to create dates in Polars
        "fecha_venta": [pl.date(random.randint(2010, 2022), random.randint(1, 12), random.randint(1, 28)) for _ in range(n)],
        "cliente_id": [random.randint(1000, 5000) for _ in range(n)]
    }
    return pl.DataFrame(data)

# Generar el DataFrame
df = generate_data()
df.head()

id_venta,producto,cantidad,precio_unitario,fecha_venta,cliente_id
i64,str,i64,f64,object,i64
1,"""dekgo""",11,43.586835,"2014-02-16 00:00:00.alias(""datetime"").strict_cast(Date).alias(""date"")",4138
2,"""sjnjg""",11,91.541314,"2015-06-23 00:00:00.alias(""datetime"").strict_cast(Date).alias(""date"")",4884
3,"""bnhco""",14,57.376142,"2015-02-20 00:00:00.alias(""datetime"").strict_cast(Date).alias(""date"")",1784
4,"""cudxa""",1,39.513228,"2019-07-21 00:00:00.alias(""datetime"").strict_cast(Date).alias(""date"")",1382
5,"""yrzth""",11,89.460785,"2015-09-09 00:00:00.alias(""datetime"").strict_cast(Date).alias(""date"")",2091


In [5]:
df = df.lazy()
df

# Limpieza de Datos

Utilizando Polars en modo Lazy, realice las siguientes operaciones de limpieza y manipulación de datos:

    1. `Filtrar` las ventas con cantidades menores a 5.
    2. `Agregar` una nueva columna llamada total_venta, que sea el resultado de multiplicar cantidad por precio_unitario.
    3. `Eliminar` la columna precio_unitario después de crear total_venta.
    4. `Agrupar` los datos por producto y calcular el total vendido y la cantidad total vendida por producto.
    5. `Ordenar` los resultados por el total vendido en orden descendente.

Todo esto lo debes de realizar en una sola linea de codigo

In [8]:
# Realizar las operaciones de limpieza y manipulación
resultado = (
    df
    .filter(pl.col("cantidad") >= 5)  # Filtrar las ventas con cantidades menores a 5
    .with_columns((pl.col("cantidad") * pl.col("precio_unitario")).alias("total_venta"))  # Agregar columna total_venta
    .drop("precio_unitario")  # Eliminar la columna precio_unitario
    .group_by("producto")  # Agrupar los datos por producto using group_by instead of groupby
    .agg([
        pl.sum("total_venta").alias("total_vendido"),  # Calcular el total vendido por producto
        pl.sum("cantidad").alias("cantidad_total_vendida")  # Calcular la cantidad total vendida por producto
    ])
    .sort("total_vendido", descending=True)  # Ordenar los resultados por el total vendido en orden descendente
)

# Ejecutar y mostrar los resultados
resultado_df = resultado.collect()
print(resultado_df)


shape: (781, 3)
┌──────────┬───────────────┬────────────────────────┐
│ producto ┆ total_vendido ┆ cantidad_total_vendida │
│ ---      ┆ ---           ┆ ---                    │
│ str      ┆ f64           ┆ i64                    │
╞══════════╪═══════════════╪════════════════════════╡
│ flozj    ┆ 1985.21488    ┆ 20                     │
│ rrfsr    ┆ 1941.097165   ┆ 20                     │
│ yvsax    ┆ 1904.127959   ┆ 20                     │
│ ejpgv    ┆ 1855.572677   ┆ 19                     │
│ bzlsi    ┆ 1852.498243   ┆ 19                     │
│ …        ┆ …             ┆ …                      │
│ rwzox    ┆ 74.487515     ┆ 7                      │
│ ccjii    ┆ 67.796229     ┆ 5                      │
│ nwyoe    ┆ 66.022442     ┆ 6                      │
│ fdbkf    ┆ 65.873692     ┆ 6                      │
│ bsowc    ┆ 58.564495     ┆ 5                      │
└──────────┴───────────────┴────────────────────────┘


# Guarda el resultado en un archivo `parquet`

Guarda el resultado en un archivo parquet que se llame `resultados.parquet`

In [9]:
# Tu codigo aqui# Guardar el resultado en un archivo parquet
resultado_df.write_parquet("resultados.parquet")


# Lee Parquet

Crea codigo para leer en polars el archivo parquet

In [10]:
# Leer el archivo Parquet en Polars
df_leido = pl.read_parquet("resultados.parquet")

# Mostrar los primeros registros del DataFrame
print(df_leido)

shape: (781, 3)
┌──────────┬───────────────┬────────────────────────┐
│ producto ┆ total_vendido ┆ cantidad_total_vendida │
│ ---      ┆ ---           ┆ ---                    │
│ str      ┆ f64           ┆ i64                    │
╞══════════╪═══════════════╪════════════════════════╡
│ flozj    ┆ 1985.21488    ┆ 20                     │
│ rrfsr    ┆ 1941.097165   ┆ 20                     │
│ yvsax    ┆ 1904.127959   ┆ 20                     │
│ ejpgv    ┆ 1855.572677   ┆ 19                     │
│ bzlsi    ┆ 1852.498243   ┆ 19                     │
│ …        ┆ …             ┆ …                      │
│ rwzox    ┆ 74.487515     ┆ 7                      │
│ ccjii    ┆ 67.796229     ┆ 5                      │
│ nwyoe    ┆ 66.022442     ┆ 6                      │
│ fdbkf    ┆ 65.873692     ┆ 6                      │
│ bsowc    ┆ 58.564495     ┆ 5                      │
└──────────┴───────────────┴────────────────────────┘
